In [1]:
import os 
from dotenv import load_dotenv
load_dotenv('/media/varun/Data/data science/main_files/.env')

True

In [2]:
import os 
import google.generativeai as genai 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

#set the environment variables
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

#load the model 
model=genai.GenerativeModel('gemini-pro')
llm = ChatGoogleGenerativeAI(model="gemini-pro")

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [55]:
from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

#load the model
llm = ChatGroq(model='llama3-70b-8192')

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm

zero shot agent - have no memory only interact once 



In [4]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [5]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [6]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [7]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [8]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [9]:
for obs in observations:
    insert_obs(obs)

In [10]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [11]:
os.environ['OPENAI_API_KEY'] =os.getenv('OPENAI_API_KEY')
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

model =ChatOpenAI(temperature=0,model = 'gpt-4o-mini')

# model.invoke("what is data science")


/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   
    handle_parsing_errors=True
)

In [13]:
input = ("What is the multiplication of the ratio between stock "
         "prices for 'ABC' and 'XYZ' in January 3rd and the ratio "
         "between the same stock prices in January the 4th?")
print(input)


What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?


In [14]:
result = agent_executor.invoke(input)



> Entering new SQL Agent Executor chain...
Thought: I need to find the stock prices for ABC and XYZ on January 3rd and 4th. Then I can calculate the ratios and multiply them. 
Action: sql_db_list_tables
Action Input: stocksThought: I need to find the stock prices for ABC and XYZ on January 3rd and 4th. Then I can calculate the ratios and multiply them. 
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/Thought: I need to find the stock prices for ABC and XYZ on January 3rd and 4th. Then I can calculate the ratios and multiply them. 
Action: sql_db_query_checker
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-03'SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date 

In [15]:
result

{'input': "What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?",
 'output': '0.0823'}

2 type of agents - conversational agent . these types have memory

In [16]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [17]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
conversational_agent.invoke("What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 10000*(1.08)^5
Observation: Answer: 14693.280768000006
Thought:Thought: Do I need to use a tool? No
AI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28. 


> Finished chain.


{'input': "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?",
 'chat_history': '',
 'output': 'The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.'}

In [19]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [20]:
#now testing the memory
conversational_agent.invoke("If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)
# here it remembers the above conversation




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: (15000 * 1.08^5) - 14693.28
Observation: Answer: 7346.641152000009
Thought:Thought: Do I need to use a tool? No
AI: You would have $7,346.64 more compared to the previous scenario.  


> Finished chain.


{'input': 'If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?',
 'chat_history': "Human: What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?\nAI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.",
 'output': 'You would have $7,346.64 more compared to the previous scenario.'}

3. type docstore agent - will have search and lookup as tools - used to search article and lookup answer

In [21]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0
  warn_deprecated(


In [22]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True
)

In [23]:
docstore_agent.invoke("What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes and his last words. 

Action: Search [Archimedes last words]

Observation: Search  is not a valid tool, try one of [Search, Lookup].
Thought:Could not parse LLM Output: It seems you're stuck! Let's break down why and how to proceed.

**The Problem:**

You're trying to search for information using a tool called "Search," but the system you're interacting with only recognizes "Search" or "Lookup."  

**The Solution:**

Simply change your command from "Search [Archimedes last words]" to **"Lookup [Archimedes last words]"**. 

This will allow the system to understand your request and hopefully provide you with the answer to Archimedes' last words. 


Let me know if you have any other questions! 

Observation: Invalid or incomplete response
Thought:Could not parse LLM Output: It seems like you're on the right track! 

Here's how to refine your approach:

1. **Use the Correct Tool:** As the feedback indicated, use "

{'input': "What were Archimedes' last words?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

self ask with search - the agent will ask questions to get and use search to get answers and this keep on going to find a best answer

In [24]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key=os.getenv('serp_api_key'))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

In [28]:
self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True,handle_parsing_errors=True)

In [29]:
self_ask_with_search.invoke("what was jackie chan doing in the 1950")



> Entering new AgentExecutor chain...
Yes. 

Follow up: How old was Jackie Chan in the 1950s? 
Intermediate answer: ['Jackie Chan ; Chan Kong-sang (陳港生) · (1954-04-07) 7 April 1954 (age 70) British Hong Kong · Joan Lin. \u200b. ( m. · 1982)\u200b · 2, including Jaycee Chan · Martial artist ...', "Bruce Lee, Jackie Chan & Donnie Yen over the years. (Starting from around late 1950's) : r/OldSchoolCool.", 'Hong Kong actor and martial artist.', 'Jackie Chan is 70 years old. When was Jackie Chan born? Wednesday, April 7, 1954. What is the nationality of Jackie Chan? This data ...', '... Jackie Chan. Nearly every young guy in Hong Kong ran out and bought himself ... Jack and Lewis Pullman, who is now also an actor. Christopher Reeve. 51 ...', '32, 29) argues, only from the 1950s onwards could film reflect upon its own rela- ... old, Jackie Chan, studying his per- formances in nine of the films he has ...', 'Chan Kong-sang (; born 7 April 1954), known professionally as Jackie Chan, is a Hon

KeyboardInterrupt: 

type of llm memory 


how to inspect the langchain functions

In [32]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [33]:
conversation = ConversationChain(
    llm=llm,
)

In [34]:
print(conversation.prompt.template)


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [35]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

types of memory 

In [36]:
#here the entire conversation is the given back to the llm - this is token high as all the
#the conversation is given to the llm 
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [37]:
conversation_buf("Good morning AI!")

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning!  The sun is shining brightly here in the digital world.  Is it a nice day where you are?  I hope you have a wonderful day planned! ☀️  \n\n'}

In [38]:

    conversation_buf("My interest here is to explore the potential of integrating Large Language Models with external knowledge")


{'input': 'My interest here is to explore the potential of integrating Large Language Models with external knowledge',
 'history': 'Human: Good morning AI!\nAI: Good morning!  The sun is shining brightly here in the digital world.  Is it a nice day where you are?  I hope you have a wonderful day planned! ☀️  \n\n',
 'response': "That's a fascinating topic! I'm eager to hear more about your thoughts.  \n\nIntegrating Large Language Models (LLMs) with external knowledge is a hot area of research right now.  It has the potential to significantly enhance the capabilities of LLMs, allowing them to access and process real-world information in a more meaningful way.\n\nFor example, imagine an LLM that could:\n\n* **Retrieve up-to-date facts and figures:** Instead of relying solely on the knowledge it was trained on, an LLM could access and incorporate information from databases, APIs, and the internet in real time.\n* **Provide more grounded and accurate responses:** By grounding its response

conversation summary buffer memory - here the previous conversation is summarized and given to the llm 

In [39]:
conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

conversation buffer window memory - here only the latest conversations is given back and the first ones are disregarded


In [40]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

agents with tools - we can use built in tools as well as custom tools we create.

In [41]:
!pip -q install duckduckgo-search

In [42]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [56]:

from langchain_community.tools import DuckDuckGoSearchRun

#here we are defining the tools 
from langchain.agents import Tool
from langchain.tools import BaseTool

search = DuckDuckGoSearchRun()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]
#we define a custom tool
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'


life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)
import random
#a random number tool
def random_num(input=""):
    return random.randint(0,5)

random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

#creating an agent
from langchain.agents import initialize_agent

tools = [search, random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)


# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)


In [59]:
conversational_agent.invoke("what is the time in london now ")



> Entering new AgentExecutor chain...
```json
{
    "action": "duckduckgo_search",
    "action_input": "current time in london"
}
```
Observation: The current local time in London is 67 minutes ahead of apparent solar time. London on the map. London is the capital of United Kingdom. Latitude: 51.51. Longitude: -0.13; Population: 8,962,000; Open London in Google Maps. Best restaurants in London #1 TANAKATSU - Japanese and sushi food The IANA time zone identifier for Central London is Europe/London. Sunday March 31 2024. Latest change: Summer time started. Switched to UTC +1 / British Summer Time (BST). The time was set forward one hour from 01:00AM to 02:00AM local time. Sunday October 27 2024. Next change: Winter time starts. Current local time in London and DST dates in 2024. Local time--:--BST . London time change . Next time change is in 3 months and 8 days, set your clock back 1 hour. London summer time (DST) in 2024. Daylight saving time 2024 in London begins at 1:00 AM on Sunda

{'input': 'what is the time in london now ',
 'chat_history': [HumanMessage(content='the time given was not correct'),
  AIMessage(content='Your clock appears to be 1.6 seconds behind the current time.')],
 'output': 'The current local time in London is not explicitly stated, but it is Sunday, March 31, 2024, and it is British Summer Time (BST), which is UTC+1.'}

In [60]:
conversational_agent("Can you give me a random number?")



> Entering new AgentExecutor chain...
```
{
    "action": "Random number",
    "action_input": "random"
}
```
Observation: 4
Thought:```
{
    "action": "Final Answer",
    "action_input": "The random number is 4"
}
```

> Finished chain.


{'input': 'Can you give me a random number?',
 'chat_history': [HumanMessage(content='the time given was not correct'),
  AIMessage(content='Your clock appears to be 1.6 seconds behind the current time.'),
  HumanMessage(content='what is the time in london now '),
  AIMessage(content='The current local time in London is not explicitly stated, but it is Sunday, March 31, 2024, and it is British Summer Time (BST), which is UTC+1.')],
 'output': 'The random number is 4'}

In [61]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The answer to the meaning of life is 42.17658"
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='the time given was not correct'),
  AIMessage(content='Your clock appears to be 1.6 seconds behind the current time.'),
  HumanMessage(content='what is the time in london now '),
  AIMessage(content='The current local time in London is not explicitly stated, but it is Sunday, March 31, 2024, and it is British Summer Time (BST), which is UTC+1.'),
  HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='The random number is 4')],
 'output': 'The answer to the meaning of life is 42.17658'}

In [63]:
#here we can modify the prompt given to the agent
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template

'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system that can help with a wide range 

In [64]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [65]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [66]:
conversational_agent("What is the meaning of life?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
```
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded, but is actually 42.17658."
}
```

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='what is the time in london now '),
  AIMessage(content='The current local time in London is not explicitly stated, but it is Sunday, March 31, 2024, and it is British Summer Time (BST), which is UTC+1.'),
  HumanMessage(content='Can you give me a random number?'),
  AIMessage(content='The random number is 4'),
  HumanMessage(content='What is the meaning of life?'),
  AIMessage(content='The answer to the meaning of life is 42.17658')],
 'output': 'The meaning of life is 42 if rounded, but is actually 42.17658.'}

In [67]:
from bs4 import BeautifulSoup
import requests

def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    stripped_content = strip_html_tags(html_content)

    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content

In [68]:
stripped_webpage('https://www.google.com')


'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchGoogle offered in:  हिन्दी বাংলা తెలుగు मराठी தமிழ் ગુજરાતી ಕನ್ನಡ മലയാളം ਪੰਜਾਬੀ AdvertisingBusiness SolutionsAbout GoogleGoogle.co.in© 2024 - Privacy - Terms   '

In [69]:
class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text

        stripped_content = strip_html_tags(html_content)
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000]
        return stripped_content

    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

page_getter = WebPageTool()

In [70]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [71]:
from langchain.prompts.chat import SystemMessagePromptTemplate
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

In [72]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [73]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.\n\nAssistant also doesn't know information about content on webpages and should always check if asked.\n\nOverall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specifi

In [75]:
conversational_agent.run("Is there an article about latest news  on https://techcrunch.com/? today")



> Entering new AgentExecutor chain...
```
{
    "action": "Get Webpage",
    "action_input": "https://techcrunch.com/"
}
```
Observation: 























TechCrunch | Startup and Technology News















































 



 


























Login





SearchStartupsVentureAppleSecurityAIAppsEventsStartup BattlefieldMoreFintechCloud ComputingLayoffsHardwareGoogleMicrosoftTransportationEVsMetaInstagramAmazonTikTokNewslettersPodcastsPartner ContentCrunchboard JobsContact UsSearch
















		Security	
Hackers shut down heating in Ukrainian city with malware, researchers say


Lorenzo Franceschi-Bicchierai











 
		Robotics	
Former Tesla humanoid head launches a robotics startup


Brian Heater





 
		Transportation	
GM’s Cruise abandons Origin robotaxi, takes $583 million charge


Kirsten Korosec





 
		AI	
UK school reprimanded for unlawful use of facial recognition technology


Paul Sawers





 
		AI	
Former Velodyne CEO’s deli

'According to the webpage, there are several articles about latest news on the webpage, including articles about hackers shutting down heating in a Ukrainian city, a new robotics startup, and more.'

In [78]:
conversational_agent.run("What are the titles of the top stories on https://www.cbsnews.com")



> Entering new AgentExecutor chain...
```json
{
    "action": "Get Webpage",
    "action_input": "https://www.cbsnews.com"
}
```
Observation: 
CBS News - Breaking news, 24/7 live streaming news & top stories
















Latest

U.S.


World


Politics


Entertainment


HealthWatch


MoneyWatch


Investigations


Crime


Space


Sports


Essentials


Brand Studio



Local News

Baltimore


Bay Area


Boston


Chicago


Colorado


Detroit


Los Angeles


Miami


Minnesota


New York


Philadelphia


Pittsburgh


Sacramento


Texas



Live

CBS News 24/7


Baltimore


Bay Area


Boston


Chicago


Colorado


Detroit


Los Angeles


Miami


Minnesota


New York


Philadelphia


Pittsburgh


Sacramento


Texas



Shows

48 Hours


60 Minutes


America Decides


CBS Evening News


CBS Mornings


CBS News Eye on America


CBS News Mornings


CBS Reports


CBS Saturday Morning


The Daily Report


The Dish


Face the Nation


Here Comes the Sun


Person to Person


Sunday Morning


The T

'The titles of the top stories on CBS News are: Secret Service director to resign after Trump shooting security lapses, Harris secures support for Democratic nomination after Biden drops out, Army Reserve punishes officers for dereliction of duty related to Maine shooting, and more.'

In [81]:
conversational_agent.run("how many articles are on the  site https://iasbaba.com/2024/07/daily-current-affairs-ias-upsc-prelims-and-mains-exam-23rd-july-2024/")



> Entering new AgentExecutor chain...
```
{
    "action": "Get Webpage",
    "action_input": "https://iasbaba.com/2024/07/daily-current-affairs-ias-upsc-prelims-and-mains-exam-23rd-july-2024/"
}
```
Observation: 







DAILY CURRENT AFFAIRS IAS | UPSC Prelims and Mains Exam – 23rd July 2024 | IASbaba














































 























SPEAK TO MENTOR - CALL NOW!




SUBSCRIBE














































 














 



Home
60 Days
Daily Free Initiatives 

TRP for UPSC Personality Test
Interview Mentorship Programme – 2023
Daily News & Analysis
Daily Current Affairs Quiz
Baba’s Explainer
TLP – Mains Answer Writing

Dedicated TLP Portal


60 Day – Rapid Revision (RaRe) Series – 2024
Monthly Magazines 

English Magazines
Hindi Magazines
Yojana & Kurukshetra Gist




Courses 

Baba’s Gurukul Course – 2025 

Gurukul Foundation
Gurukul Advanced


UPSC 2025 Mains 

TLP Connect – 2025
TLP (+) Plus – 2025


Integrated Learning Progr

"There are 2 articles on the site: IASbaba's Daily Current Affairs Analysis, and BHIL PRADESH."